In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn.metrics import ndcg_score
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import lightgbm as lgb
import numpy as np
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path
from scipy.stats import hmean
import scipy
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LassoCV



In [3]:
"""
just drop the path of your new data directory nothing more than that. 

"""

path = r'C:\Users\Family\Downloads\widsdatathon2025 (3)'

def read_data(base_path:str) -> pd.DataFrame :
    path = Path(base_path)
    trc=pd.read_excel(path /'TRAIN_NEW'  / 'TRAIN_CATEGORICAL_METADATA_new.xlsx')
    trq=pd.read_excel(path /'TRAIN_NEW'  / 'TRAIN_QUANTITATIVE_METADATA_new.xlsx')
    trf=pd.read_csv(path   /'TRAIN_NEW'  / 'TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')
    trs=pd.read_excel(path /'TRAIN_NEW'  / 'TRAINING_SOLUTIONS.xlsx')  
    tsc=pd.read_excel(path /'TEST'      / 'TEST_CATEGORICAL.xlsx')
    tsq=pd.read_excel(path /'TEST'       / 'TEST_QUANTITATIVE_METADATA.xlsx')    
    tsf=pd.read_csv(path   /'TEST'       / 'TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')    
    sub=pd.read_excel(path / 'SAMPLE_SUBMISSION.xlsx')    
    dic=pd.read_excel(path /'Data Dictionary.xlsx')
    return trc, trq, trf, trs, tsc, tsq, tsf, sub, dic

trc, trq, trf, trs, tsc, tsq, tsf, sub, dic = read_data(base_path=path)

# Data Merging 
cq = pd.merge(trc, trq, on='participant_id', how='left')
feat = pd.merge(cq, trf, on='participant_id', how='left')  
qc = pd.merge(tsc, tsq, on='participant_id', how='left')
train = pd.merge(feat, trs, on='participant_id', how='left') 
test = pd.merge(qc, tsf, on='participant_id', how='left')
train_sex =train

In [4]:
train_ids = train['participant_id']
test_ids = test['participant_id'] # I will store them for later usage in grouping in validation why?  I don't want the same user to appear in both train and test. 
num_feats = trq # numerical features
cat_feats = trc # seperate categorical and numerical features help me reteriving them later easily for preprocessing.
target_cols = ['ADHD_Outcome', 'Sex_F']
groups = train_ids



In [5]:
'''
Train has 25 features/columns with missing values: ['PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race', 'MRI_Track_Scan_Location', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ', 'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan']
Test  has 23 features/columns with missing values: ['PreInt_Demos_Fam_Child_Ethnicity', 'PreInt_Demos_Fam_Child_Race', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ', 'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial']
fMRI has no missing values
Extra columns in train: ['MRI_Track_Age_at_Scan', 'MRI_Track_Scan_Location']
'''

# Find columns with missing values only
train_missing_features_to_impute = train.columns[train.isnull().any()].tolist() # List of features with missing values in train, only 25 and no missing data in fMRI data
test_missing_features_to_impute = test.columns[test.isnull().any()].tolist() # List of features with missing values in test, only 23 and no missing data in fMRI data


# Initialize the imputer
imputer = IterativeImputer(estimator=LassoCV(random_state=42), max_iter=5, random_state=42)

# Impute in-place for train
if train_missing_features_to_impute:
	train[train_missing_features_to_impute] = imputer.fit_transform(train[train_missing_features_to_impute])

# Impute in-place for test
if test_missing_features_to_impute:
	test[test_missing_features_to_impute] = imputer.fit_transform(test[test_missing_features_to_impute])

c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.220e+00, tolerance: 4.923e-01
  model = cd_fast.enet_coordinate_descent(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.402e+00, tolerance: 5.646e-01
  model = cd_fast.enet_coordinate_descent(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing reg

In [6]:
train.isnull().sum().sum(), test.isnull().sum().sum() # Check if there are any missing values left in train and test data

(0, 0)

In [7]:
train

,participant_id,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ,...,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn,ADHD_Outcome,Sex_F
0,00aIpNTbG5uh,2019,4,1.0,0.0,3.0,21.0,45.000000,17.903324,32.871543,...,-0.280312,0.037560,0.423037,0.242453,0.336213,0.402338,0.327915,0.539032,1,0
1,00fV0OyyoLfw,2017,1,0.0,9.0,2.0,21.0,0.000000,21.000000,45.000000,...,-0.332783,-0.332711,0.556939,0.475578,0.429196,0.457970,0.312571,0.595978,1,0
2,04X1eiS79T4B,2017,1,1.0,2.0,2.0,9.0,0.000000,12.387745,17.897575,...,-0.002132,-0.175586,0.679183,0.290292,0.486680,0.255208,0.575017,0.605182,0,1
3,05ocQutkURd6,2018,1,3.0,8.0,2.0,18.0,10.000000,18.000000,0.000000,...,-0.199576,-0.216457,0.519074,0.298586,0.415466,0.511607,0.361204,0.446613,0,1
4,06YUNBA9ZRLq,2018,1,0.0,1.0,2.0,12.0,0.000000,13.845904,22.432518,...,-0.141012,-0.002865,0.515169,0.336139,0.316430,0.442230,0.177079,0.378278,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,zwjJWCRzKhDz,2019,4,1.0,1.0,3.0,12.0,15.136452,15.000000,5.000000,...,0.112789,0.211312,0.601190,0.587116,0.312695,0.485938,0.189102,0.354333,0,1
1209,zwXD5v17Rx01,2018,1,0.0,0.0,3.0,21.0,40.000000,21.000000,40.000000,...,0.253990,0.198741,0.648260,0.055241,0.491985,0.118676,0.404331,0.537121,1,0
1210,zWzLCi3NTBTd,2018,3,2.0,3.0,3.0,21.0,40.000000,21.000000,35.000000,...,0.044653,0.234887,0.538475,0.024265,0.472322,0.095624,0.205326,0.182633,1,1
1211,Zy9GTHDxUbXU,2019,4,0.0,1.0,3.0,18.0,35.000000,18.000000,45.000000,...,-0.035955,-0.062152,0.706214,0.183288,0.104987,0.420463,0.152727,0.706737,1,0


In [8]:
"""# Convert all categorical features to strings (to avoid mixed types)
for feature in trc.columns:
    train[feature] = train[feature].astype(object)
for feature in cat_feats:
    train[feature] = train[feature].astype(str)
    test[feature] = test[feature].astype(str)

# One-Hot Encoding for categorical features
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

for feature in cat_feats:
    if feature not in train.columns or feature not in test.columns:  # Skip if the feature is missing
        continue

    # Apply OneHotEncoder
    train_encoded = encoder.fit_transform(train[[feature]])
    test_encoded = encoder.transform(test[[feature]])

    # Convert encoded features to DataFrame and append them to the original data
    train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out([feature]))
    test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out([feature]))

    # Drop the original feature and concatenate the new encoded columns
    train = pd.concat([train.drop(columns=[feature]), train_encoded_df], axis=1)
    test = pd.concat([test.drop(columns=[feature]), test_encoded_df], axis=1)"""

"# Convert all categorical features to strings (to avoid mixed types)\nfor feature in trc.columns:\n    train[feature] = train[feature].astype(object)\nfor feature in cat_feats:\n    train[feature] = train[feature].astype(str)\n    test[feature] = test[feature].astype(str)\n\n# One-Hot Encoding for categorical features\nencoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)\n\nfor feature in cat_feats:\n    if feature not in train.columns or feature not in test.columns:  # Skip if the feature is missing\n        continue\n\n    # Apply OneHotEncoder\n    train_encoded = encoder.fit_transform(train[[feature]])\n    test_encoded = encoder.transform(test[[feature]])\n\n    # Convert encoded features to DataFrame and append them to the original data\n    train_encoded_df = pd.DataFrame(train_encoded, columns=encoder.get_feature_names_out([feature]))\n    test_encoded_df = pd.DataFrame(test_encoded, columns=encoder.get_feature_names_out([feature]))\n\n    # Drop the original 

In [9]:
scaler = StandardScaler()

# Only apply scaling to numerical columns that are not part of the target or categorical features
numerical_features = [col for col in train.columns if col not in target_cols and col not in cat_feats]

# Fit scaler on the numerical features of the train set and transform train and test sets
train[numerical_features] = scaler.fit_transform(train[numerical_features])  # Fit and transform for train set
test[numerical_features] = scaler.transform(test[numerical_features]) 

In [10]:
train.isnull().sum().sum(), test.isnull().sum().sum() # Check if there are any missing values left in train and test data

(0, 0)

In [11]:
y_sex = train['Sex_F']  
y_adhd = train['ADHD_Outcome']  


# feature importance in sex

In [12]:
train

,participant_id,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ,...,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn,ADHD_Outcome,Sex_F
0,00aIpNTbG5uh,2019,4,1.0,0.0,3.0,21.0,45.000000,17.903324,32.871543,...,-1.185011,0.141726,-0.890926,-0.677929,-0.180457,-0.554190,-0.835889,-0.024380,1,0
1,00fV0OyyoLfw,2017,1,0.0,9.0,2.0,21.0,0.000000,21.000000,45.000000,...,-1.441008,-1.721379,-0.001464,0.499990,0.305932,-0.239836,-0.922257,0.310269,1,0
2,04X1eiS79T4B,2017,1,1.0,2.0,2.0,9.0,0.000000,12.387745,17.897575,...,0.172175,-0.930766,0.810557,-0.436211,0.606624,-1.385552,0.554995,0.364357,0,1
3,05ocQutkURd6,2018,1,3.0,8.0,2.0,18.0,10.000000,18.000000,0.000000,...,-0.791115,-1.136420,-0.252985,-0.394303,0.234110,0.063241,-0.648517,-0.567490,0,1
4,06YUNBA9ZRLq,2018,1,0.0,1.0,2.0,12.0,0.000000,13.845904,22.432518,...,-0.505395,-0.061681,-0.278927,-0.204557,-0.283942,-0.328775,-1.684912,-0.969062,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,zwjJWCRzKhDz,2019,4,1.0,1.0,3.0,12.0,15.136452,15.000000,5.000000,...,0.732856,1.016003,0.292482,1.063558,-0.303480,-0.081801,-1.617241,-1.109779,0,1
1209,zwXD5v17Rx01,2018,1,0.0,0.0,3.0,21.0,40.000000,21.000000,40.000000,...,1.421749,0.952747,0.605150,-1.623857,0.634374,-2.157031,-0.405763,-0.035614,1,0
1210,zWzLCi3NTBTd,2018,3,2.0,3.0,3.0,21.0,40.000000,21.000000,35.000000,...,0.400430,1.134626,-0.124110,-1.780372,0.531519,-2.287286,-1.525919,-2.118786,1,1
1211,Zy9GTHDxUbXU,2019,4,0.0,1.0,3.0,18.0,35.000000,18.000000,45.000000,...,0.007159,-0.359999,0.990114,-0.976870,-1.389992,-0.451771,-1.821987,0.961151,1,0


In [13]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

# Assuming 'train' and 'test' are your DataFrames
# and target_cols is defined as ['ADHD_Outcome', 'Sex_F']

# Prepare features (X) - drop targets and participant_id
X = train.drop(columns=target_cols + ['participant_id'])

# Convert categorical features to dummy variables
X = pd.get_dummies(X, drop_first=True)

# Encode target (y_sex should be your target Series)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_sex)

# Initialize and fit the LGBM model for Sex prediction
model_sex = lgb.LGBMClassifier(
    class_weight='balanced',  # Important for imbalanced data
    random_state=42
)
model_sex.fit(X, y)

# Get feature importances and create a DataFrame
importance_df_sex = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model_sex.feature_importances_
}).sort_values('Importance', ascending=False)

# Filter features with importance > 0 (or set a higher threshold)
important_features_sex = importance_df_sex[importance_df_sex['Importance'] > 0]['Feature'].tolist()

# Filter the data - ensure test has same features
train_sex = train[important_features_sex]
test_sex = test[important_features_sex]

# Verify the filtered data
print(f"\nNumber of important features for Sex: {len(important_features_sex)}")
print("\nTop 10 important features for Sex prediction:")
print(importance_df_sex.head(10))

[LightGBM] [Info] Number of positive: 416, number of negative: 797
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.574558 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5075431
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.574558 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5075431
[LightGBM] [Info] Number of data points in the train set: 1213, number of used features: 19927
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Info] Number of data points in the train set: 1213, number of used features: 19927
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000

Number of important features for Sex: 2310

Top 10 important features for Se

In [ ]:
# Instead of > 0 threshold, consider:
threshold = importance_df_sex['Importance'].quantile(0.98)  
important_features_sex = importance_df_sex[importance_df_sex['Importance'] > threshold]['Feature'].tolist()

In [15]:
important_features_sex=train[important_features_sex]

In [16]:
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

# Initialize storage
oof_sex = np.zeros(len(important_features_sex))
models_sep_sex = {}  # Initialize models_sep_sex as an empty dictionary
bag_f1_sex = []

BAGS = 5
FOLDS = 5

# Prepare data
important_features_sex_filtered = important_features_sex.reset_index(drop=True)
y_sex = y_sex.reset_index(drop=True)

for bag in range(BAGS):
    print('#'*25)
    print(f'### Bag {bag+1}')
    print('#'*25)
    
    models_sep_sex[bag] = []
    bag_pred_sex = np.zeros(len(important_features_sex))
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=bag*FOLDS)
    fold_f1_sex = []
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X=important_features_sex_filtered, y=y_sex)):
        print(f'=> Fold {fold+1}, ', end='')

        # Dynamic class balancing
        y_important_features_sex = y_sex.iloc[train_idx]
        class_counts = y_important_features_sex.value_counts()
        majority_class = class_counts.idxmax()
        minority_class = 1 - majority_class
        
        majority_samples = y_important_features_sex[y_important_features_sex == majority_class]
        minority_samples = y_important_features_sex[y_important_features_sex == minority_class]
        
        n_samples = len(minority_samples)
        if len(majority_samples) < n_samples:
            n_samples = len(majority_samples)
        
        downsampled_majority = majority_samples.sample(
            n=n_samples, 
            replace=len(majority_samples) < len(minority_samples),
            random_state=bag*BAGS+fold
        )
        
        sex_train_idx = minority_samples.index.union(downsampled_majority.index)
        X_important_features_sex = important_features_sex_filtered.iloc[sex_train_idx]
        y_important_features_sex_balanced = y_sex.iloc[sex_train_idx]
        X_valid_sex = important_features_sex_filtered.iloc[valid_idx]
        y_valid_sex = y_sex.iloc[valid_idx]

        # Model parameters
        model_params = {
            'n_estimators': 300,
            'learning_rate': 0.175, 
            'metric': 'cross_entropy_lambda',
            'objective': None,
            'boosting_type': 'gbdt',
            'num_leaves': 31,
            'reg_alpha': 0.0,
            'reg_lambda': 0.0009,
            'early_stopping_round': 50,
            'feature_fraction': 0.1,
            'subsample': 0.8,
            'verbose': -1
        }

        model_sep_sex = LGBMClassifier(**model_params)
        model_sep_sex.fit(X_important_features_sex, y_important_features_sex_balanced,
                          eval_set=[(X_valid_sex, y_valid_sex)])

        sex_pred = model_sep_sex.predict(X_valid_sex)
        fold_f1_sex.append(f1_score(y_valid_sex, sex_pred, average='macro'))
        
        bag_pred_sex[valid_idx] = model_sep_sex.predict_proba(X_valid_sex)[:, 1]
        models_sep_sex[bag].append(model_sep_sex)

    bag_avg_f1_sex = np.mean(fold_f1_sex)
    bag_f1_sex.append(bag_avg_f1_sex)
    print(f"\nBag {bag+1} Avg F1 - Sex: {bag_avg_f1_sex:.4f}")
    oof_sex += bag_pred_sex / BAGS

# Final evaluation
final_f1_sex = f1_score(y_sex, (oof_sex > 0.5).astype(int), average='macro')
print('\n' + '#'*50)
print(f"Final Ensemble F1 - Sex: {final_f1_sex:.4f}")
print(f"Mean Bag F1 - Sex: {np.mean(bag_f1_sex):.4f} ± {np.std(bag_f1_sex):.4f}")


#########################
### Bag 1
#########################
=> Fold 1, => Fold 2, => Fold 2, => Fold 3, => Fold 3, => Fold 4, => Fold 4, => Fold 5, => Fold 5, 
Bag 1 Avg F1 - Sex: 0.7486
#########################
### Bag 2
#########################
=> Fold 1, 
Bag 1 Avg F1 - Sex: 0.7486
#########################
### Bag 2
#########################
=> Fold 1, => Fold 2, => Fold 2, => Fold 3, => Fold 3, => Fold 4, => Fold 4, => Fold 5, => Fold 5, 
Bag 2 Avg F1 - Sex: 0.7508
#########################
### Bag 3
#########################
=> Fold 1, 
Bag 2 Avg F1 - Sex: 0.7508
#########################
### Bag 3
#########################
=> Fold 1, => Fold 2, => Fold 2, => Fold 3, => Fold 3, => Fold 4, => Fold 4, => Fold 5, => Fold 5, 
Bag 3 Avg F1 - Sex: 0.7543
#########################
### Bag 4
#########################
=> Fold 1, 
Bag 3 Avg F1 - Sex: 0.7543
#########################
### Bag 4
#########################
=> Fold 1, => Fold 2, => Fold 2, => Fold 3, => Fold 3, => Fold 4, =>

# ADHD USING LGBM

In [31]:
# Drop only the columns from `trf` that are present in `train`
columns_to_drop = [col for col in trf.columns if col in train.columns]
train_adhd = train.drop(columns=columns_to_drop)

# Drop the target columns
train_adhd = train_adhd.drop(columns=target_cols)

## LGBM WITH DOWNSAMPLING 

In [28]:
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from scipy.stats import hmean

# Initialize storage
oof_predictions = np.zeros(len(train))
models = {}
bag_f1_scores = []
stds = []
F1s = []

BAGS = 5
FOLDS = 5
target_col = 'ADHD_Outcome'

# Prepare data
X = train.drop(columns=[target_col, 'participant_id', 'Sex_F'])
X = pd.get_dummies(X, drop_first=True)
y = train[target_col]

for bag in range(BAGS):
    print('#'*25)
    print(f'### Bag {bag+1}')
    print('#'*25)
    
    models[bag] = []
    bag_pred = np.zeros(len(X))
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=bag*FOLDS)
    fold_f1s = []
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y), 1):
        print(f'=> Fold {fold}, ', end='')

        # Your exact downsampling technique
        y_train = y.iloc[train_idx]
        class_counts = y_train.value_counts()
        
        if len(class_counts) > 1:
            majority_class = class_counts.idxmax()
            minority_class = 1 - majority_class
            majority_samples = y_train[y_train == majority_class]
            minority_samples = y_train[y_train == minority_class]
            
            n_samples = len(minority_samples)
            downsampled_majority = majority_samples.sample(
                n=n_samples, 
                replace=len(majority_samples) < len(minority_samples),
                random_state=bag*BAGS+fold
            )
            balanced_idx = minority_samples.index.union(downsampled_majority.index)
            X_train = X.iloc[balanced_idx]
            y_train_balanced = y.iloc[balanced_idx]
        else:
            X_train = X.iloc[train_idx]
            y_train_balanced = y.iloc[train_idx]

        # Model training (unchanged)
        model = LGBMClassifier(
            n_estimators=300,
            learning_rate=0.175,
            num_leaves=31,
            reg_alpha=0.0,
            reg_lambda=0.0009,
            feature_fraction=0.1,
            subsample=0.8,
            verbose=-1,
            class_weight='balanced'
        )
        model.fit(X_train, y_train_balanced)
        
        # Get predictions
        pred = model.predict(X.iloc[valid_idx])
        fold_f1 = f1_score(y.iloc[valid_idx], pred)
        fold_f1s.append(fold_f1)
        print(f"F1: {fold_f1:.4f}")
        
        # Store predictions
        bag_pred[valid_idx] = model.predict_proba(X.iloc[valid_idx])[:, 1]
        models[bag].append(model)
        
        # Store validation set std for this fold
        stds.append(y.iloc[valid_idx].std())
    
    # Bag-level results
    bag_avg_f1 = np.mean(fold_f1s)
    bag_f1_scores.append(bag_avg_f1)
    print(f"\nBag {bag+1} Avg F1: {bag_avg_f1:.4f}")
    oof_predictions += bag_pred / BAGS
    F1s.append(fold_f1s)

# Convert to numpy array for calculations
F1s = np.array(F1s)

# Final evaluation (aligned with your original validation structure)
print('\n' + '#'*50)
print("==== Overall Results ===")
print(f"Mean F1: {np.mean(F1s):.4f}")
print(f"F1 std: {np.std(F1s):.4f}")

print("\nStandard deviations of test sets:")
for i, std in enumerate(stds):
    print(f"Fold {i+1}: {std:.4f}")

# Final OOF score calculation
final_preds = (oof_predictions > 0.5).astype(int)
final_f1 = f1_score(y, final_preds)
print(f"\nFinal OOF F1: {final_f1:.4f}")

#########################
### Bag 1
#########################
=> Fold 1, F1: 0.8013
=> Fold 2, F1: 0.8013
=> Fold 2, F1: 0.7682
=> Fold 3, F1: 0.7682
=> Fold 3, F1: 0.7810
=> Fold 4, F1: 0.7810
=> Fold 4, F1: 0.8037
=> Fold 5, F1: 0.8037
=> Fold 5, F1: 0.8287

Bag 1 Avg F1: 0.7966
#########################
### Bag 2
#########################
=> Fold 1, F1: 0.8287

Bag 1 Avg F1: 0.7966
#########################
### Bag 2
#########################
=> Fold 1, F1: 0.7834
=> Fold 2, F1: 0.7834
=> Fold 2, F1: 0.8113
=> Fold 3, F1: 0.8113
=> Fold 3, F1: 0.7987
=> Fold 4, F1: 0.7987
=> Fold 4, F1: 0.7582
=> Fold 5, F1: 0.7582
=> Fold 5, F1: 0.8202

Bag 2 Avg F1: 0.7944
#########################
### Bag 3
#########################
=> Fold 1, F1: 0.8202

Bag 2 Avg F1: 0.7944
#########################
### Bag 3
#########################
=> Fold 1, F1: 0.7974
=> Fold 2, F1: 0.7974
=> Fold 2, F1: 0.8091
=> Fold 3, F1: 0.8091
=> Fold 3, F1: 0.7925
=> Fold 4, F1: 0.7925
=> Fold 4, F1: 0.7821
=> Fold 

## LGBM WITHOUT DOWNSAMPLING

In [29]:
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

# Initialize storage
oof_predictions = np.zeros(len(train))
models = {}
bag_f1_scores = []
stds = []
F1s = []

BAGS = 5
FOLDS = 5
target_col = 'ADHD_Outcome'

# Prepare data
X = train.drop(columns=[target_col, 'participant_id', 'Sex_F'])
X = pd.get_dummies(X, drop_first=True)
y = train[target_col]

for bag in range(BAGS):
    print('#'*25)
    print(f'### Bag {bag+1}')
    print('#'*25)
    
    models[bag] = []
    bag_pred = np.zeros(len(X))
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=bag*FOLDS)
    fold_f1s = []
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y), 1):
        print(f'=> Fold {fold}, ', end='')

        # Model training WITHOUT downsampling
        model = LGBMClassifier(
            n_estimators=300,
            learning_rate=0.175,
            num_leaves=31,
            reg_alpha=0.0,
            reg_lambda=0.0009,
            feature_fraction=0.1,
            subsample=0.8,
            verbose=-1,
            class_weight='balanced'  # Handling imbalance through class weights
        )
        model.fit(X.iloc[train_idx], y.iloc[train_idx])
        
        # Get predictions
        pred = model.predict(X.iloc[valid_idx])
        fold_f1 = f1_score(y.iloc[valid_idx], pred)
        fold_f1s.append(fold_f1)
        print(f"F1: {fold_f1:.4f}")
        
        # Store predictions
        bag_pred[valid_idx] = model.predict_proba(X.iloc[valid_idx])[:, 1]
        models[bag].append(model)
        
        # Store validation set std for this fold
        stds.append(y.iloc[valid_idx].std())
    
    # Bag-level results
    bag_avg_f1 = np.mean(fold_f1s)
    bag_f1_scores.append(bag_avg_f1)
    print(f"\nBag {bag+1} Avg F1: {bag_avg_f1:.4f}")
    oof_predictions += bag_pred / BAGS
    F1s.append(fold_f1s)

# Final evaluation
print('\n' + '#'*50)
print("==== Overall Results ===")
print(f"Mean F1 across bags: {np.mean(bag_f1_scores):.4f} ± {np.std(bag_f1_scores):.4f}")

print("\nFold-wise standard deviations:")
for i, std in enumerate(stds[:FOLDS]):  # Show first fold's std as representative
    print(f"Fold {i+1}: {std:.4f}")

# Final OOF score
final_preds = (oof_predictions > 0.5).astype(int)
final_f1 = f1_score(y, final_preds)
print(f"\nFinal OOF F1: {final_f1:.4f}")

#########################
### Bag 1
#########################
=> Fold 1, F1: 0.8453
=> Fold 2, F1: 0.8453
=> Fold 2, F1: 0.8385
=> Fold 3, F1: 0.8385
=> Fold 3, F1: 0.8689
=> Fold 4, F1: 0.8689
=> Fold 4, F1: 0.8556
=> Fold 5, F1: 0.8556
=> Fold 5, F1: 0.8501

Bag 1 Avg F1: 0.8517
#########################
### Bag 2
#########################
=> Fold 1, F1: 0.8501

Bag 1 Avg F1: 0.8517
#########################
### Bag 2
#########################
=> Fold 1, F1: 0.8478
F1: 0.8478
=> Fold 2, => Fold 2, F1: 0.8315
=> Fold 3, F1: 0.8315
=> Fold 3, F1: 0.8500
F1: 0.8500
=> Fold 4, => Fold 4, F1: 0.8453
=> Fold 5, F1: 0.8453
=> Fold 5, F1: 0.8595
F1: 0.8595

Bag 2 Avg F1: 0.8468
#########################
### Bag 3
#########################
=> Fold 1, 
Bag 2 Avg F1: 0.8468
#########################
### Bag 3
#########################
=> Fold 1, F1: 0.8485
=> Fold 2, F1: 0.8485
=> Fold 2, F1: 0.8476
=> Fold 3, F1: 0.8476
=> Fold 3, F1: 0.8289
=> Fold 4, F1: 0.8289
=> Fold 4, F1: 0.8338
=> Fold 

## ONLY TRAIN_ADHD

In [33]:
train_adhd

,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ,EHQ_EHQ_Total,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,2019,4,1.0,0.0,3.0,21.0,45.000000,17.903324,32.871543,0.818599,...,0.459262,0.737479,0.776497,0.823227,0.320435,0.880342,0.400556,-0.132149,0.603281,1.052928
1,2017,1,0.0,9.0,2.0,21.0,0.000000,21.000000,45.000000,0.662301,...,1.451997,1.199700,0.776497,1.309451,0.320435,0.880342,0.686599,0.348525,0.123977,1.088973
2,2017,1,1.0,2.0,2.0,9.0,0.000000,12.387745,17.897575,0.549071,...,0.459262,1.815994,2.167026,0.580116,2.108335,0.522620,2.688906,2.271221,-0.355327,0.773597
3,2018,1,3.0,8.0,2.0,18.0,10.000000,18.000000,0.000000,0.683936,...,-1.029841,-1.111402,-1.077543,-1.121665,-1.467465,-0.908267,-0.743619,-0.132149,-0.834631,-0.566931
4,2018,1,0.0,1.0,2.0,12.0,0.000000,13.845904,22.432518,-1.203370,...,1.948365,1.661920,2.167026,1.795674,1.393175,1.238063,0.972643,-0.612823,-1.793238,-1.572278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,2019,4,1.0,1.0,3.0,12.0,15.136452,15.000000,5.000000,0.818599,...,1.451997,1.199700,1.240007,1.309451,0.320435,0.880342,0.686599,-0.132149,0.123977,0.018928
1209,2018,1,0.0,0.0,3.0,21.0,40.000000,21.000000,40.000000,-0.259717,...,-0.037106,-1.111402,-1.077543,-0.635442,-1.467465,-0.908267,-1.315707,-1.093497,-0.355327,-1.327849
1210,2018,3,2.0,3.0,3.0,21.0,40.000000,21.000000,35.000000,-2.012158,...,-0.037106,0.275259,0.312987,0.337004,-0.394725,0.522620,0.114512,-0.132149,0.603281,-0.432709
1211,2019,4,0.0,1.0,3.0,18.0,35.000000,18.000000,45.000000,0.818599,...,-0.533473,-0.341035,-1.077543,0.093893,0.320435,0.522620,-0.743619,-0.132149,-0.355327,-0.994939


In [39]:
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier

# Initialize storage
oof_predictions = np.zeros(len(train_adhd))
models = {}
bag_f1_scores = []
stds = []
F1s = []

BAGS = 5
FOLDS = 5
target_col = 'ADHD_Outcome'

# Prepare data
X = train_adhd
X = pd.get_dummies(X, drop_first=True)
y = train[target_col]

for bag in range(BAGS):
    print('#'*25)
    print(f'### Bag {bag+1}')
    print('#'*25)
    
    models[bag] = []
    bag_pred = np.zeros(len(X))
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=bag*FOLDS)
    fold_f1s = []
    
    for fold, (train_adhd_idx, valid_idx) in enumerate(skf.split(X, y), 1):
        print(f'=> Fold {fold}, ', end='')

        # Model train_adhding WITHOUT downsampling
        model = LGBMClassifier(
            n_estimators=100,
            learning_rate=0.175,
            num_leaves=31,
            reg_alpha=0.0,
            reg_lambda=0.0009,
            feature_fraction=0.1,
            subsample=0.8,
            verbose=-1,
            class_weight='balanced'  # Handling imbalance through class weights
        )
        model.fit(X.iloc[train_adhd_idx], y.iloc[train_adhd_idx])
        
        # Get predictions
        pred = model.predict(X.iloc[valid_idx])
        fold_f1 = f1_score(y.iloc[valid_idx], pred)
        fold_f1s.append(fold_f1)
        print(f"F1: {fold_f1:.4f}")
        
        # Store predictions
        bag_pred[valid_idx] = model.predict_proba(X.iloc[valid_idx])[:, 1]
        models[bag].append(model)
        
        # Store validation set std for this fold
        stds.append(y.iloc[valid_idx].std())
    
    # Bag-level results
    bag_avg_f1 = np.mean(fold_f1s)
    bag_f1_scores.append(bag_avg_f1)
    print(f"\nBag {bag+1} Avg F1: {bag_avg_f1:.4f}")
    oof_predictions += bag_pred / BAGS
    F1s.append(fold_f1s)

# Final evaluation
print('\n' + '#'*50)
print("==== Overall Results ===")
print(f"Mean F1 across bags: {np.mean(bag_f1_scores):.4f} ± {np.std(bag_f1_scores):.4f}")

print("\nFold-wise standard deviations:")
for i, std in enumerate(stds[:FOLDS]):  # Show first fold's std as representative
    print(f"Fold {i+1}: {std:.4f}")

# Final OOF score
final_preds = (oof_predictions > 0.5).astype(int)
final_f1 = f1_score(y, final_preds)
print(f"\nFinal OOF F1: {final_f1:.4f}")

#########################
### Bag 1
#########################
=> Fold 1, F1: 0.8363
=> Fold 2, F1: 0.7768
=> Fold 3, F1: 0.8480
=> Fold 4, F1: 0.8448
=> Fold 5, F1: 0.8343

Bag 1 Avg F1: 0.8280
#########################
### Bag 2
#########################
=> Fold 1, F1: 0.7977
=> Fold 2, F1: 0.8131
=> Fold 3, F1: 0.8443
=> Fold 4, F1: 0.8503
=> Fold 5, F1: 0.8496

Bag 2 Avg F1: 0.8310
#########################
### Bag 3
#########################
=> Fold 1, F1: 0.8204
=> Fold 2, F1: 0.8446
=> Fold 3, F1: 0.8276
=> Fold 4, F1: 0.8353
=> Fold 5, F1: 0.8354

Bag 3 Avg F1: 0.8326
#########################
### Bag 4
#########################
=> Fold 1, F1: 0.8094
=> Fold 2, F1: 0.8174
=> Fold 3, F1: 0.8319
=> Fold 4, F1: 0.8546
=> Fold 5, F1: 0.8494

Bag 4 Avg F1: 0.8325
#########################
### Bag 5
#########################
=> Fold 1, F1: 0.8314
=> Fold 2, F1: 0.8408
=> Fold 3, F1: 0.8300
=> Fold 4, F1: 0.8319
=> Fold 5, F1: 0.8085

Bag 5 Avg F1: 0.8285

#############################

In [41]:
from scipy.stats import hmean

# Using harmonic mean of bag scores (though mathematically similar for single metric)
final_f1_hmean = hmean(bag_f1_scores)  
print(f"Final ADHD F1 (harmonic mean): {final_f1_hmean:.4f}")

Final ADHD F1 (harmonic mean): 0.8305


# SEX USING LOGISTIC

In [18]:
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import brier_score_loss, f1_score
import numpy as np

# Define variables
FOLDS = 5
REPEATS = 1
SEED = fold * REPEATS

def eval_metrics(y_true, y_pred, label="None", thresh=0.16):
    """Evaluate predictions using Brier Score and F1 Score."""
    brier = brier_score_loss(y_true, y_pred)
    f1 = f1_score(y_true, (y_pred > thresh).astype(int))
    print(f"{label} -> Brier Score: {brier:.4f}, F1: {f1:.4f}")
    return brier, f1

# Store OOF Brier and F1
scores_sex = []
sex_oof = np.zeros(len(y_sex))

# Classification threshold
t_sex = 0.3

# CV Setup
rskf = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=REPEATS, random_state=SEED)
skf = StratifiedKFold(n_splits=FOLDS)

params_2 = {
    "penalty": "l1",
    "Cs": 10,
    "cv": skf,
    "fit_intercept": True,
    "scoring": "f1",
    "random_state": SEED,
    "solver": "saga"
}

model_2 = LogisticRegressionCV(**params_2)

# Align indices of important_features_sex and y_sex
important_features_sex = important_features_sex.reset_index(drop=True)
y_sex = y_sex.reset_index(drop=True)

for fold, (train_idx, val_idx) in enumerate(rskf.split(important_features_sex, y_sex), 1):
    print(f"\n=== Fold {fold} ===")

    # Split data
    X_train, X_val = important_features_sex.iloc[train_idx], important_features_sex.iloc[val_idx]
    y_important_features_sex, y_val_sex = y_sex.iloc[train_idx], y_sex.iloc[val_idx]

    # Train model on sex
    model_2.fit(X_train, y_important_features_sex)

    # Predict
    sex_val = model_2.predict_proba(X_val)[:, 1]
    sex_oof[val_idx] += sex_val / REPEATS

    # Evaluate
    sex_brier, sex_f1 = eval_metrics(y_val_sex, sex_val, label="Sex", thresh=t_sex)
    scores_sex.append((sex_brier, sex_f1))

# Final CV Results
print(f"\n=== CV Results ===")
print(f"Sex Mean Brier Score: {np.mean([s[0] for s in scores_sex]):.4f}")
print(f"Sex Mean F1: {np.mean([s[1] for s in scores_sex]):.4f}")



=== Fold 1 ===


c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\my

Sex -> Brier Score: 0.1517, F1: 0.7253

=== Fold 2 ===


c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\my

Sex -> Brier Score: 0.1670, F1: 0.6774

=== Fold 3 ===


...

c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\my

Sex -> Brier Score: 0.2013, F1: 0.6630

=== Fold 5 ===


c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Family\Desktop\WIDS-Com\my

Sex -> Brier Score: 0.1757, F1: 0.6961

=== CV Results ===
Sex Mean Brier Score: 0.1743
Sex Mean F1: 0.6857


c:\Users\Family\Desktop\WIDS-Com\myvenv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


# ADHD using Logistic Regression

In [19]:
# Drop only the columns from `trf` that are present in `train`
columns_to_drop = [col for col in trf.columns if col in train.columns]
train_adhd = train.drop(columns=columns_to_drop)

# Drop the target columns
train_adhd = train_adhd.drop(columns=target_cols)

In [20]:
train_adhd

,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ,EHQ_EHQ_Total,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,2019,4,1.0,0.0,3.0,21.0,45.000000,17.903324,32.871543,0.818599,...,0.459262,0.737479,0.776497,0.823227,0.320435,0.880342,0.400556,-0.132149,0.603281,1.052928
1,2017,1,0.0,9.0,2.0,21.0,0.000000,21.000000,45.000000,0.662301,...,1.451997,1.199700,0.776497,1.309451,0.320435,0.880342,0.686599,0.348525,0.123977,1.088973
2,2017,1,1.0,2.0,2.0,9.0,0.000000,12.387745,17.897575,0.549071,...,0.459262,1.815994,2.167026,0.580116,2.108335,0.522620,2.688906,2.271221,-0.355327,0.773597
3,2018,1,3.0,8.0,2.0,18.0,10.000000,18.000000,0.000000,0.683936,...,-1.029841,-1.111402,-1.077543,-1.121665,-1.467465,-0.908267,-0.743619,-0.132149,-0.834631,-0.566931
4,2018,1,0.0,1.0,2.0,12.0,0.000000,13.845904,22.432518,-1.203370,...,1.948365,1.661920,2.167026,1.795674,1.393175,1.238063,0.972643,-0.612823,-1.793238,-1.572278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,2019,4,1.0,1.0,3.0,12.0,15.136452,15.000000,5.000000,0.818599,...,1.451997,1.199700,1.240007,1.309451,0.320435,0.880342,0.686599,-0.132149,0.123977,0.018928
1209,2018,1,0.0,0.0,3.0,21.0,40.000000,21.000000,40.000000,-0.259717,...,-0.037106,-1.111402,-1.077543,-0.635442,-1.467465,-0.908267,-1.315707,-1.093497,-0.355327,-1.327849
1210,2018,3,2.0,3.0,3.0,21.0,40.000000,21.000000,35.000000,-2.012158,...,-0.037106,0.275259,0.312987,0.337004,-0.394725,0.522620,0.114512,-0.132149,0.603281,-0.432709
1211,2019,4,0.0,1.0,3.0,18.0,35.000000,18.000000,45.000000,0.818599,...,-0.533473,-0.341035,-1.077543,0.093893,0.320435,0.522620,-0.743619,-0.132149,-0.355327,-0.994939


In [21]:

features_adhd = ['Basic_Demos_Enroll_Year', 'Barratt_Barratt_P1_Edu',
       'Barratt_Barratt_P2_Edu', 'EHQ_EHQ_Total', 'ColorVision_CV_Score',
       'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV',
       'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP',
       'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total',
       'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing',
       'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity',
       'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial',
       'MRI_Track_Age_at_Scan']

# Features to be interacted with predicted probability of Sex_F = 1
interactions = [
    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity",
    "MRI_Track_Age_at_Scan", "SDQ_SDQ_Generating_Impact"
]

In [22]:
train_adhd

,Basic_Demos_Enroll_Year,Basic_Demos_Study_Site,PreInt_Demos_Fam_Child_Ethnicity,PreInt_Demos_Fam_Child_Race,MRI_Track_Scan_Location,Barratt_Barratt_P1_Edu,Barratt_Barratt_P1_Occ,Barratt_Barratt_P2_Edu,Barratt_Barratt_P2_Occ,EHQ_EHQ_Total,...,SDQ_SDQ_Conduct_Problems,SDQ_SDQ_Difficulties_Total,SDQ_SDQ_Emotional_Problems,SDQ_SDQ_Externalizing,SDQ_SDQ_Generating_Impact,SDQ_SDQ_Hyperactivity,SDQ_SDQ_Internalizing,SDQ_SDQ_Peer_Problems,SDQ_SDQ_Prosocial,MRI_Track_Age_at_Scan
0,2019,4,1.0,0.0,3.0,21.0,45.000000,17.903324,32.871543,0.818599,...,0.459262,0.737479,0.776497,0.823227,0.320435,0.880342,0.400556,-0.132149,0.603281,1.052928
1,2017,1,0.0,9.0,2.0,21.0,0.000000,21.000000,45.000000,0.662301,...,1.451997,1.199700,0.776497,1.309451,0.320435,0.880342,0.686599,0.348525,0.123977,1.088973
2,2017,1,1.0,2.0,2.0,9.0,0.000000,12.387745,17.897575,0.549071,...,0.459262,1.815994,2.167026,0.580116,2.108335,0.522620,2.688906,2.271221,-0.355327,0.773597
3,2018,1,3.0,8.0,2.0,18.0,10.000000,18.000000,0.000000,0.683936,...,-1.029841,-1.111402,-1.077543,-1.121665,-1.467465,-0.908267,-0.743619,-0.132149,-0.834631,-0.566931
4,2018,1,0.0,1.0,2.0,12.0,0.000000,13.845904,22.432518,-1.203370,...,1.948365,1.661920,2.167026,1.795674,1.393175,1.238063,0.972643,-0.612823,-1.793238,-1.572278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1208,2019,4,1.0,1.0,3.0,12.0,15.136452,15.000000,5.000000,0.818599,...,1.451997,1.199700,1.240007,1.309451,0.320435,0.880342,0.686599,-0.132149,0.123977,0.018928
1209,2018,1,0.0,0.0,3.0,21.0,40.000000,21.000000,40.000000,-0.259717,...,-0.037106,-1.111402,-1.077543,-0.635442,-1.467465,-0.908267,-1.315707,-1.093497,-0.355327,-1.327849
1210,2018,3,2.0,3.0,3.0,21.0,40.000000,21.000000,35.000000,-2.012158,...,-0.037106,0.275259,0.312987,0.337004,-0.394725,0.522620,0.114512,-0.132149,0.603281,-0.432709
1211,2019,4,0.0,1.0,3.0,18.0,35.000000,18.000000,45.000000,0.818599,...,-0.533473,-0.341035,-1.077543,0.093893,0.320435,0.522620,-0.743619,-0.132149,-0.355327,-0.994939


In [23]:
# Actual Validation and Modeling
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score, f1_score
import numpy as np

# Constants
SEED = 42
REPEATS = 5
FOLDS = 5
t_sex = 0.3
t_adhd = 0.4

# Evaluation function
def eval_metrics(y_true, y_pred, weights, label="None", thresh=0.3):
    """Evaluate predictions using F1 Score and ROC-AUC."""
    roc_auc = roc_auc_score(y_true, y_pred)
    f1 = f1_score(y_true, (y_pred > thresh).astype(int), sample_weight=weights)
    print(f"{label} -> F1: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")
    return f1, roc_auc

# Initialize lists to store scores and out-of-fold predictions
scores_sex = []
scores_adhd = []
sex_oof = np.zeros(len(y_sex))
adhd_oof = np.zeros(len(y_adhd))

# Cross-validation setup
rskf = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=REPEATS, random_state=SEED)
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

# Logistic RegressionCV parameters
logreg_params = {
    "penalty": "l2",
    "cv": skf,
    "fit_intercept": True,
    "scoring": "f1",
    "random_state": SEED,
    "solver": "saga"
}

model_sex = LogisticRegressionCV(**logreg_params)
model_adhd = LogisticRegressionCV(**logreg_params)

# Start cross-validation loop
for fold, (train_idx, val_idx) in enumerate(rskf.split(train_adhd, y_adhd), 1):
    print(f"\n=== Fold {fold} ===")

    # Split data
    X_train, X_val = train_adhd.iloc[train_idx].copy(), train_adhd.iloc[val_idx].copy()
    y_train_sex, y_val_sex = y_sex.iloc[train_idx], y_sex.iloc[val_idx]
    y_train_adhd, y_val_adhd = y_adhd.iloc[train_idx], y_adhd.iloc[val_idx]

    # Sample weights
    weights_train = np.where(y_train_adhd == "11", 2, 1)  # Assign higher weight to ADHD positive samples
    weights_val = np.where(y_val_adhd == "11", 2, 1)  # Assign higher weight to ADHD positive samples

    # Train model to predict Sex_F
    model_sex.fit(X_train, y_train_sex, sample_weight=weights_train)
    sex_train_pred = model_sex.predict_proba(X_train)[:, 1]
    sex_val_pred = model_sex.predict_proba(X_val)[:, 1]
    sex_oof[val_idx] += sex_val_pred / REPEATS

    sex_f1, sex_roc_auc = eval_metrics(y_val_sex, sex_val_pred, weights_val, "Sex_F", thresh=t_sex)
    scores_sex.append((sex_f1, sex_roc_auc))

    # Add predicted sex probabilities
    X_train = X_train.assign(sex_proba=sex_train_pred)
    X_val = X_val.assign(sex_proba=sex_val_pred)

    # Create interaction features
    for col in interactions:
        X_train[f"I_{col}"] = X_train[col] * X_train["sex_proba"]
        X_val[f"I_{col}"] = X_val[col] * X_val["sex_proba"]

    # Train model to predict ADHD outcome
    model_adhd.fit(X_train[features_adhd], y_train_adhd, sample_weight=weights_train)
    adhd_val_pred = model_adhd.predict_proba(X_val[features_adhd])[:, 1]
    adhd_oof[val_idx] += adhd_val_pred / REPEATS

    adhd_f1, adhd_roc_auc = eval_metrics(y_val_adhd, adhd_val_pred, weights_val, "Outcome ADHD", thresh=t_adhd)
    scores_adhd.append((adhd_f1, adhd_roc_auc))

# Print final results
print(f"\n=== CV Results ===")
print(f"Sex Mean roc auc Score: {np.mean([b for b, _ in scores_sex]):.4f}")
print(f"Sex Mean F1: {np.mean([f for _, f in scores_sex]):.4f}")
print(f"ADHD Mean roc auc Score: {np.mean([b for b, _ in scores_adhd]):.4f}")
print(f"ADHD Mean F1: {np.mean([f for _, f in scores_adhd]):.4f}")


=== Fold 1 ===
Sex_F -> F1: 0.5139, ROC-AUC: 0.6287
Sex_F -> F1: 0.5139, ROC-AUC: 0.6287
Outcome ADHD -> F1: 0.8146, ROC-AUC: 0.8465

=== Fold 2 ===
Outcome ADHD -> F1: 0.8146, ROC-AUC: 0.8465

=== Fold 2 ===
Sex_F -> F1: 0.4946, ROC-AUC: 0.5423
Sex_F -> F1: 0.4946, ROC-AUC: 0.5423
Outcome ADHD -> F1: 0.8117, ROC-AUC: 0.8256

=== Fold 3 ===
Outcome ADHD -> F1: 0.8117, ROC-AUC: 0.8256

=== Fold 3 ===
Sex_F -> F1: 0.5357, ROC-AUC: 0.6349
Sex_F -> F1: 0.5357, ROC-AUC: 0.6349
Outcome ADHD -> F1: 0.8117, ROC-AUC: 0.7383

=== Fold 4 ===
Outcome ADHD -> F1: 0.8117, ROC-AUC: 0.7383

=== Fold 4 ===
Sex_F -> F1: 0.5235, ROC-AUC: 0.6267
Sex_F -> F1: 0.5235, ROC-AUC: 0.6267
Outcome ADHD -> F1: 0.8137, ROC-AUC: 0.8133

=== Fold 5 ===
Outcome ADHD -> F1: 0.8137, ROC-AUC: 0.8133

=== Fold 5 ===
Sex_F -> F1: 0.4453, ROC-AUC: 0.4910
Sex_F -> F1: 0.4453, ROC-AUC: 0.4910
Outcome ADHD -> F1: 0.8137, ROC-AUC: 0.7506

=== Fold 6 ===
Outcome ADHD -> F1: 0.8137, ROC-AUC: 0.7506

=== Fold 6 ===
Sex_F -> F1: 0

## only features_adhd

In [24]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import brier_score_loss, f1_score
import numpy as np

FOLDS = 5
SEED = 42

# Initialize storage
scores_f1 = []
scores_brier = []

# Store OOF predictions
adhd_oof = np.zeros(len(y_adhd))

# Classification threshold
t_adhd = 0.4

# Stratified K-Fold
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

params = {
    "penalty": "l1",
    "cv": skf,
    "fit_intercept": True,
    "scoring": "f1",
    "random_state": SEED,
    "solver": "saga"
    }

model = LogisticRegressionCV(**params)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_adhd, y_adhd), 1):
    print(f"\n=== Fold {fold} ===")

    # Data split
    X_train, X_val = train_adhd.iloc[train_idx], train_adhd.iloc[val_idx]
    y_train = y_adhd.iloc[train_idx]
    y_val = y_adhd.iloc[val_idx]
    
    # Sample weights as in the first script
    weights_train = ((y_sex.iloc[train_idx] == 1) & (y_train == 1)).astype(int) + 1
    weights_val = ((y_sex.iloc[val_idx] == 1) & (y_val == 1)).astype(int) + 1

    # Column check
    missing = [col for col in features_adhd if col not in X_train.columns]
    if missing:
        print(f"Missing columns: {missing}")
    features_adhd_valid = [col for col in features_adhd if col in X_train.columns]

    # Fit and predict
    model.fit(X_train[features_adhd_valid], y_train, sample_weight=weights_train)
    pred_proba = model.predict_proba(X_val[features_adhd_valid])[:, 1]
    adhd_oof[val_idx] = pred_proba

    # Calculate scores
    brier = brier_score_loss(y_val, pred_proba)
    f1 = f1_score(y_val, (pred_proba > t_adhd).astype(int), sample_weight=weights_val)

    print(f"F1 Score: {f1:.4f}, Brier Score: {brier:.4f}")

    scores_f1.append(f1)
    scores_brier.append(brier)

# ===== Results Summary =====

scores_f1 = np.array(scores_f1)
scores_brier = np.array(scores_brier)

print("\n=== Final Evaluation ===")
print(f"Mean F1: {scores_f1.mean():.4f}")
print(f"Std F1: {scores_f1.std():.4f}")
print(f"Mean Brier: {scores_brier.mean():.4f}")
print(f"Std Brier: {scores_brier.std():.4f}")



=== Fold 1 ===
F1 Score: 0.8498, Brier Score: 0.2172

=== Fold 2 ===
F1 Score: 0.8498, Brier Score: 0.2172

=== Fold 2 ===
F1 Score: 0.8469, Brier Score: 0.2192

=== Fold 3 ===
F1 Score: 0.8469, Brier Score: 0.2192

=== Fold 3 ===
F1 Score: 0.8481, Brier Score: 0.2192

=== Fold 4 ===
F1 Score: 0.8481, Brier Score: 0.2192

=== Fold 4 ===
F1 Score: 0.8527, Brier Score: 0.2177

=== Fold 5 ===
F1 Score: 0.8527, Brier Score: 0.2177

=== Fold 5 ===
F1 Score: 0.8516, Brier Score: 0.2177

=== Final Evaluation ===
Mean F1: 0.8498
Std F1: 0.0021
Mean Brier: 0.2182
Std Brier: 0.0008
F1 Score: 0.8516, Brier Score: 0.2177

=== Final Evaluation ===
Mean F1: 0.8498
Std F1: 0.0021
Mean Brier: 0.2182
Std Brier: 0.0008


## all adhd data

In [25]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import brier_score_loss, f1_score
import numpy as np

FOLDS = 5
SEED = 42

# Initialize storage
scores_f1 = []
scores_brier = []

# Store OOF predictions
adhd_oof = np.zeros(len(y_adhd))

# Classification threshold
t_adhd = 0.4

# Stratified K-Fold
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

params = {
    "penalty": "l1",
    "cv": skf,
    "fit_intercept": True,
    "scoring": "f1",
    "random_state": SEED,
    "solver": "saga"
}

model = LogisticRegressionCV(**params)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_adhd, y_adhd), 1):
    print(f"\n=== Fold {fold} ===")

    # Data split
    X_train, X_val = train_adhd.iloc[train_idx], train_adhd.iloc[val_idx]
    y_train = y_adhd.iloc[train_idx]
    y_val = y_adhd.iloc[val_idx]
    
    # Sample weights as in the first script
    weights_train = ((y_sex.iloc[train_idx] == 1) & (y_train == 1)).astype(int) + 1
    weights_val = ((y_sex.iloc[val_idx] == 1) & (y_val == 1)).astype(int) + 1

    
    # Fit and predict
    model.fit(X_train, y_train, sample_weight=weights_train)
    pred_proba = model.predict_proba(X_val)[:, 1]
    adhd_oof[val_idx] = pred_proba

    # Calculate scores
    brier = brier_score_loss(y_val, pred_proba)
    f1 = f1_score(y_val, (pred_proba > t_adhd).astype(int), sample_weight=weights_val)

    print(f"F1 Score: {f1:.4f}, Brier Score: {brier:.4f}")

    scores_f1.append(f1)
    scores_brier.append(brier)

# ===== Results Summary =====

scores_f1 = np.array(scores_f1)
scores_brier = np.array(scores_brier)

print("\n=== Final Evaluation ===")
print(f"Mean F1: {scores_f1.mean():.4f}")
print(f"Std F1: {scores_f1.std():.4f}")
print(f"Mean Brier: {scores_brier.mean():.4f}")
print(f"Std Brier: {scores_brier.std():.4f}")



=== Fold 1 ===
F1 Score: 0.8498, Brier Score: 0.2172

=== Fold 2 ===
F1 Score: 0.8498, Brier Score: 0.2172

=== Fold 2 ===
F1 Score: 0.8469, Brier Score: 0.1961

=== Fold 3 ===
F1 Score: 0.8469, Brier Score: 0.1961

=== Fold 3 ===
F1 Score: 0.8481, Brier Score: 0.2192

=== Fold 4 ===
F1 Score: 0.8481, Brier Score: 0.2192

=== Fold 4 ===
F1 Score: 0.8527, Brier Score: 0.2177

=== Fold 5 ===
F1 Score: 0.8527, Brier Score: 0.2177

=== Fold 5 ===
F1 Score: 0.8516, Brier Score: 0.2177

=== Final Evaluation ===
Mean F1: 0.8498
Std F1: 0.0021
Mean Brier: 0.2136
Std Brier: 0.0088
F1 Score: 0.8516, Brier Score: 0.2177

=== Final Evaluation ===
Mean F1: 0.8498
Std F1: 0.0021
Mean Brier: 0.2136
Std Brier: 0.0088


## prediction

In [26]:
# Define train_sex_filtered to match the features used during training
train_sex_filtered = train_sex[important_features_sex]

# Predict the Sex target using the trained models
sex_pred = np.zeros(len(test_clean))

for bag in range(BAGS):
    for fold in range(FOLDS):
        # Get the trained model for this bag and fold
        model_sex = models_sep_sex[bag][fold]
        
        # Ensure test_clean contains only the features used during training
        test_features_sex = train_sex_filtered.columns  # Use the same columns as in training for Sex
        test_clean_sex = test_clean[test_features_sex]

        # Make predictions for the test set
        sex_pred += model_sex.predict_proba(test_clean_sex)[:, 1] / (BAGS * FOLDS)

# Convert probabilities to binary predictions
sex_pred_binary = (sex_pred > 0.5).astype(int)


ValueError: Boolean array expected for the condition, not float64

In [ ]:
# Ensure test_clean is preprocessed similarly to train_clean
test_clean_preprocessed = test_clean.fillna(test_clean.mean())

# Predict ADHD outcomes using the pipeline
adhd_pred = model_sep_adhd.predict(test_clean_preprocessed)
adhd_pred_binary = (adhd_pred > 0.5).astype(int)

In [ ]:
# Ensure all arrays have the same length
if len(id_test) == len(sex_pred_binary):
    # Create a DataFrame for submission
    submission = pd.DataFrame({
        'participant_id': id_test,
         'ADHD_Outcome' : adhd_pred_binary ,
        'Sex_F': sex_pred_binary  
    })

    # Save the submission file
    submission.to_csv('submission_Sex_0.7416_adhd_0.76417.csv', index=False)
    print("Submission file for Sex created successfully!")

Submission file for Sex created successfully!
